## Investigating problems closing a deposit account

GitHub: https://github.com/MkershMambu/MambuAPINotebook/blob/master/CloseDepositAccount.ipynb

Viewer: [https://nbviewer.jupyter.org/github/MkershMambu/MambuAPINotebook/blob/master/CloseDepositAccount.ipynb](https://nbviewer.jupyter.org/github/MkershMambu/MambuAPINotebook/blob/master/CloseDepositAccount.ipynb?flush_cache=true)

In [2]:
%run API.py
%run ENV.py
from IPython.core.display import HTML
setENV(ENV)

### First lets create a new deposit account

In [92]:
%%writefile JSONBODY
{
  "overdraftSettings": {
	"allowOverdraft": true,
	"overdraftLimit": 0
  },
  "accountType": "CURRENT_ACCOUNT",
  "name": "MKCurTest222",
  "accountHolderKey": "8a8186ac692678910169288509c606bc",
  "productTypeKey": "8a8186aa69491c25016949888aeb10d2",
  "currencyCode": "EUR",
  "accountHolderType": "CLIENT"
}

Overwriting JSONBODY


In [93]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
r = POST('{{env1}}/deposits', headers=HEADERS, params=PARAMS, body="JSONBODY")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)
encodedKey = r.json()['encodedKey']

API status:
201
Now the JSON:
{"encodedKey":"8a8187176a121bc2016a128877a501a4","creationDate":"2019-04-12T19:11:34+02:00","lastModifiedDate":"2019-04-12T19:11:34+02:00","id":"VBHX672","name":"MKCurTest222","accountHolderType":"CLIENT","accountHolderKey":"8a8186ac692678910169288509c606bc","accountState":"PENDING_APPROVAL","productTypeKey":"8a8186aa69491c25016949888aeb10d2","accountType":"CURRENT_ACCOUNT","currencyCode":"EUR","assignedBranchKey":"8a8186ac692678910169287cf43606af","internalControls":{},"overdraftSettings":{"allowOverdraft":true,"overdraftLimit":0},"interestSettings":{"interestPaymentSettings":{"interestPaymentDates":[]}},"overdraftInterestSettings":{"interestRateSettings":{"encodedKey":"8a8187176a121bc2016a128877a501a5","interestRate":0E-20,"interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIXED","interestRateSource":"FIXED_INTEREST_RATE"}},"balances":{"overdraftAmount":0,"technicalOverdraftAmount":0,"locke

### Deposit money into the account

In [94]:
%%writefile JSONBODY
{
  "transactionDetails": {
    "transactionChannelId": "8a818e74677a2e9201677ec2b4c336a6"
  },
  "amount": 500,
  "notes": "some notes",
  "paymentOrderId": "MK1204_1",
  "externalId": "MK1204_1"
}

Overwriting JSONBODY


In [95]:

HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}/deposit-transactions".format(encodedKey)
r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
400
Now the JSON:
{"errors":[{"errorCode":3305,"errorReason":"EXTERNAL_ID_ALREADY_EXISTS"}]}


### We need to approve the deposit account before we can deposit into it

In [96]:
%%writefile JSONBODY
{
  "action": "APPROVE",
  "notes": "more notes"
}

Overwriting JSONBODY


In [97]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}:changeState".format(encodedKey)
r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
200
Now the JSON:
{"encodedKey":"8a8187176a121bc2016a128877a501a4","creationDate":"2019-04-12T19:11:34+02:00","lastModifiedDate":"2019-04-12T19:11:49+02:00","id":"VBHX672","name":"MKCurTest222","accountHolderType":"CLIENT","accountHolderKey":"8a8186ac692678910169288509c606bc","accountState":"APPROVED","productTypeKey":"8a8186aa69491c25016949888aeb10d2","accountType":"CURRENT_ACCOUNT","approvedDate":"2019-04-12T19:11:49+02:00","currencyCode":"EUR","assignedBranchKey":"8a8186ac692678910169287cf43606af","internalControls":{},"overdraftSettings":{"allowOverdraft":true,"overdraftLimit":0E-10},"interestSettings":{"interestPaymentSettings":{"interestPaymentDates":[]}},"overdraftInterestSettings":{"interestRateSettings":{"encodedKey":"8a8187176a121bc2016a128877a501a5","interestRate":0E-20,"interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIXED","interestRateSource":"FIXED_INTEREST_RATE"}},"balances":{"overdraftAmount

### Now we should be able to deposit

In [100]:
%%writefile JSONBODY
{
  "transactionDetails": {
    "transactionChannelId": "8a818e74677a2e9201677ec2b4c336a6"
  },
  "amount": 500,
  "notes": "some notes",
  "paymentOrderId": "MK1204_6",
  "externalId": "MK1204_7"
}

Overwriting JSONBODY


In [101]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}/deposit-transactions".format(encodedKey)
r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
201
Now the JSON:
{"encodedKey":"8a8187176a121bc2016a128877a501a8","id":"92","externalId":"MK1204_7","paymentOrderId":"MK1204_6","creationDate":"2019-04-12T19:12:14+02:00","valueDate":"2019-04-12T19:12:14+02:00","notes":"some notes","parentAccountKey":"8a8187176a121bc2016a128877a501a4","type":"DEPOSIT","amount":500,"currencyCode":"EUR","affectedAmounts":{"fundsAmount":500.0000000000,"interestAmount":0,"feesAmount":0,"overdraftAmount":0E-10,"overdraftFeesAmount":0E-10,"overdraftInterestAmount":0E-10,"technicalOverdraftAmount":0E-10,"technicalOverdraftInterestAmount":0E-10,"fractionAmount":0},"taxes":{},"accountBalances":{"totalBalance":500.0000000000},"userKey":"8a8186ac69267891016928db604e0735","branchKey":"8a8186ac692678910169287cf43606af","terms":{"interestSettings":{},"overdraftInterestSettings":{"interestRate":0E-20},"overdraftSettings":{"overdraftLimit":0E-10}},"transactionDetails":{"transactionChannelKey":"8a818e74677a2e9201677ec2b4c336a6","transactionChannelId":"cash

### Now let's see if we can DELETE

In [102]:

HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
r = DELETE(url, headers=HEADERS, params=PARAMS)
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
400
Now the JSON:
{"errors":[{"errorCode":105,"errorReason":"INVALID_ACCOUNT_STATE"}]}


### Get Account in a state where we can DELETE

You can't delete whilst there are funds in the account. So Let's change balance to 0

In [103]:
%%writefile JSONBODY
{
  "transactionDetails": {
    "transactionChannelId": "8a818e74677a2e9201677ec2b4c336a6"
  },
  "amount": 500,
  "notes": "some notes",
  "paymentOrderId": "MK1204_9",
  "externalId": "MK1204_91"
}

Overwriting JSONBODY


In [104]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}/withdrawal-transactions".format(encodedKey)
r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
201
Now the JSON:
{"encodedKey":"8a8186656a125bbb016a126f4dc80058","id":"93","externalId":"MK1204_91","paymentOrderId":"MK1204_9","creationDate":"2019-04-12T19:12:35+02:00","valueDate":"2019-04-12T19:12:35+02:00","notes":"some notes","parentAccountKey":"8a8187176a121bc2016a128877a501a4","type":"WITHDRAWAL","amount":-500,"currencyCode":"EUR","affectedAmounts":{"fundsAmount":500.0000000000,"interestAmount":0,"feesAmount":0,"overdraftAmount":0E-10,"overdraftFeesAmount":0,"overdraftInterestAmount":0,"technicalOverdraftAmount":0E-10,"technicalOverdraftInterestAmount":0,"fractionAmount":0},"taxes":{},"accountBalances":{"totalBalance":0E-10},"userKey":"8a8186ac69267891016928db604e0735","branchKey":"8a8186ac692678910169287cf43606af","terms":{"interestSettings":{},"overdraftInterestSettings":{},"overdraftSettings":{}},"transactionDetails":{"transactionChannelKey":"8a818e74677a2e9201677ec2b4c336a6","transactionChannelId":"cash"},"transferDetails":{},"fees":[]}


### Now let's see if we can DELETE now

In [105]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
r = DELETE(url, headers=HEADERS, params=PARAMS)
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
400
Now the JSON:
{"errors":[{"errorCode":105,"errorReason":"INVALID_ACCOUNT_STATE"}]}


No we can't DELETE. Once account has been approved and had activity on it you can no longer DELETE.
See [https://support.mambu.com/docs/deposit-accounts-life-cycle-and-states](https://support.mambu.com/docs/deposit-accounts-life-cycle-and-states)

We can certainly close from the Mambu UI and this changes "accountState": "CLOSED".

Can we change state to closed using "/deposits/{depositID}:changeState" function?
* No the only actions that you can pass to this are (APPROVE, UNDO_APPROVE, LOCK) See [here](https://api.mambu.com/#tocSdepositaccountaction)
* Can we update the account (with a PUT). No See below

In [60]:
%%writefile JSONBODY
{
    "accountHolderKey": "8a8186ac692678910169288509c606bc",
    "accountHolderType": "CLIENT",
    "accountState": "CLOSED",
    "accountType": "CURRENT_ACCOUNT",
    "accruedAmounts": {
        "interestAccrued": 0.0,
        "overdraftInterestAccrued": 0.0,
        "technicalOverdraftInterestAccrued": 0.0
    },
    "activationDate": "2019-04-12T18:44:57+02:00",
    "approvedDate": "2019-04-12T18:44:22+02:00",
    "assignedBranchKey": "8a8186ac692678910169287cf43606af",
    "balances": {
        "availableBalance": 0.0,
        "feesDue": 0.0,
        "holdBalance": 0.0,
        "lockedBalance": 0.0,
        "overdraftAmount": 0.0,
        "overdraftInterestDue": 0.0,
        "technicalOverdraftAmount": 0.0,
        "technicalOverdraftInterestDue": 0.0
    },
    "creationDate": "2019-04-12T18:44:05+02:00",
    "currencyCode": "EUR",
    "encodedKey": "8a8186656a125bbb016a126f4dc80055",
    "id": "BSGX467",
    "interestSettings": {
        "interestPaymentSettings": {
            "interestPaymentDates": []
        }
    },
    "internalControls": {},
    "lastModifiedDate": "2019-04-12T18:45:39+02:00",
    "name": "MKCurTest222",
    "overdraftInterestSettings": {
        "interestRateSettings": {
            "encodedKey": "8a8186656a125bbb016a126f4dc80056",
            "interestChargeFrequency": "ANNUALIZED",
            "interestChargeFrequencyCount": 1,
            "interestRate": 0.0,
            "interestRateSource": "FIXED_INTEREST_RATE",
            "interestRateTerms": "FIXED",
            "interestRateTiers": []
        }
    },
    "overdraftSettings": {
        "allowOverdraft": true,
        "overdraftLimit": 0.0
    },
    "productTypeKey": "8a8186aa69491c25016949888aeb10d2"
}

Overwriting JSONBODY


In [61]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
r = PUT(url, headers=HEADERS, params=PARAMS, body="JSONBODY")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
400
Now the JSON:
{"errors":[{"errorCode":2813,"errorSource":"The account state cannot be changed","errorReason":"DEPOSIT_ACCOUNT_FIELD_NOT_EDITABLE"}]}


### Can we PATCH to make the change

In [79]:
%%writefile JSONBODY
[
  {
    "op": "REPLACE",
    "path": "accountState",
    "value": "CLOSED"
  }
]

Overwriting JSONBODY


In [80]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
r = PATCH(url, headers=HEADERS, params=PARAMS, body="JSONBODY")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
400
Now the JSON:
{"errors":[{"errorCode":2001,"errorSource":"Field cannot be patched: /accountState","errorReason":"FIELD_NOT_ALLOWED"}]}


Testing that my PATCH works for other attributes

It does I tested with the following:

In [77]:
%%writefile JSONBODY
[
  {
    "op": "REPLACE",
    "path": "name",
    "value": "MKCurTest222YYYYY"
  }
]

Overwriting JSONBODY


### Is there anything in API 1.0 to do this 
[https://support.mambu.com/docs/savings-products-api](https://support.mambu.com/docs/savings-products-api)

Nothing obvious. Let's to try a DELETE in API 1.0 to see if it behaves differently thena API 2.0

In [106]:
HEADERS = {}
PARAMS = {}
url = "{{env1}}" + "/savings/{0}".format(encodedKey)
r = DELETE(url, headers=HEADERS, params=PARAMS)
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
400
Now the JSON:
{"returnCode":416,"returnStatus":"INVALID_SAVINGS_ACCOUNT_STATE_TRANSITION"}


### Call to check the current state of the Account

In [91]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json'}
# Parameters below set up pagination and only returns max 20
# url = "{{env1}}/deposits/NSUX690"
url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
PARAMS = {'detailsLevel': 'FULL', 'paginationDetails': 'ON','limit': 20}
r = GET(url, headers=HEADERS, params=PARAMS)
PRINT(r)

API status: 404
JSON Response:
{
    "errors": [
        {
            "errorCode": 428,
            "errorReason": "INVALID_DEPOSIT_ACCOUNT_ID"
        }
    ]
}
